# Music Recommendation by Using LightGBM

## WSDM - KKBox's Music Recommendation Challenge

In this final project, we are going to create a music recommendation system. The problem link is [here](https://www.kaggle.com/c/kkbox-music-recommendation-challenge).

In this task, we will predict the chances of a user listening to a song repetitively after the first observable listening event within a time window was triggered. If there are recurring listening event(s) triggered within a month after the user’s very first observable listening event, its target is marked 1, and 0 otherwise in the training set. The same rule applies to the testing set.

KKBOX provides a training data set consists of information of the first observable listening event for each unique user-song pair within a specific time duration. Metadata of each unique user and song pair is also provided. The use of public data to increase the level of accuracy of your prediction is encouraged.

The train and the test data are selected from users listening history in a given time period. Note that this time period is chosen to be before the WSDM-KKBox Churn Prediction time period. The train and test sets are split based on time, and the split of public/private are based on unique user/song pairs.

## Boosting Machine Learning

In this project, we adopt boosting machine learning technique.

The philosophy of boosting is to use a set of __weak learnners__ to create a __strong learner__. A __weak learner__ is defined to be a classifier which is only slightly correlated with the true classification. In contrast, a __strong learner__ is a classifier that is arbitrarily well-correlated with the true classification.

Boosting algorithms consist of iteratively training weak learners with respect to a distribution and adding them to a final strong learner. When the weak learners are added, they are typically weighted with respect to their accuracy. Every time after a weak learner is added, the data are reweighted: __examples that are misclassified gain weight and examples that are classified correctly lose weight__. In this way, future weak learners will focus more on the examples that previous weak learners misclassified. Then by adding new weak learners again and again, finally we will get a strong learner that could handle the task much better.

## Light Gradient Boosting Machine (LightGBM)

When the weak learner is chosen as (small fixed size) __decision tree__, and differentiable loss functions are adopted, we get a __Gradient Boosting Decision Tree (GBDT)__ [1]. Due to its efficiency, accuracy, and interpretability, GBDT achieves state-of-the-art performances in many machine learning tasks, such as multi-class classification, click prediction, and learning to rank.

However, __conventional implementations__ of GBDT need to, for every feature, scan all the data instances to estimate the information gain of all the possible split points. This makes these implementations very time consuming when handling big data. To this end, in 2017 NIPS, Microsoft published a __new implementation__ of GBDT, called __LightGBM__, which speeds up the training process of conventional GBDT by up to over 20 times while achieving almost the same accuracy in experiments on multiple public datasets [1].

LightGBM adopted two novel techiques to try to achieve better performance:

1. Gradient-based One-Side Sampling (GOSS). According to the definition of information gain, instances with larger gradients (under-trained instances) will contribute more to the information gain. So when down sampling the data instances, one could better keep those instances with large gradients, and only randomly drop those instances with small gradients. It is proved that such a treatment can lead to a more accurate gain estimation than uniformly random sampling, with the same target sampling rate.

2. Exclusive Feature Bundling (EFB). In real applications, usually the feature space is quite sparse, many features are (almost) exclusive, i.e., they rarely take nonzero values simultaneously. So it is safely to bundle such exclusive features. To this end, LightGBM includes an efficient algorithm by reducing the optimal bundling problem to a graph coloring problem, and solving it by a greedy algorithm with a constant approximation ratio.

With the reduction of size of training data and reduction of number of features, LightGBM is equipped with the ability of faster training speed and higher efficiency. 

## Code Realization

Microsoft made its LightGBM package open source. The package can be found and installed from Github [4].

Our code is based on [here](https://www.kaggle.com/takeshikagawa/calc-roc-area). We make changes to that in following aspects:

1. **The original training dataset provided by Kaggle is too large for our slow computers to work on. So does the test dataset. Furthermore, the test data provided by Kaggle does not have labels. This makes us unable to see how the algorithm works. Therefore, we decide to discard the test data. Instead, we extract 10,000 samples from the training data, and split it into a larger group with 7000 samples for training and a smaller group with 3000 samples for testing.**

2. **We compared the LightGBM model with random guessing.**

In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from tqdm import tqdm

In [3]:
ntr = 7000
nts = 3000
print('Loading data...')
data_path = './data/'
train = pd.read_csv(data_path + 'train.csv',nrows=ntr)
names=['msno','song_id','source_system_tab','source_screen_name',\
      'source_type','target']
test1 = pd.read_csv(data_path+'train.csv',names=names,skiprows=ntr,nrows=nts)
songs = pd.read_csv(data_path + 'songs.csv')
members = pd.read_csv(data_path + 'members.csv')

Loading data...


## Data Analysis



### train.csv

* msno: user id
* song_id: song id
* source_system_tab: the name of the tab where the event was triggered. System tabs are used to categorize KKBOX mobile apps functions. For example, tab my library contains functions to manipulate the local storage, and tab search contains functions relating to search.
* source_screen_name: name of the layout a user sees.
* source_type: an entry point a user first plays music on mobile apps. An entry point could be album, online-playlist, song .. etc.
* target: this is the target variable. target=1 means there are recurring listening event(s) triggered within a month after the user’s very first observable listening event, target=0 otherwise .

In [4]:
train.head(6)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1
5,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,explore,Explore,online-playlist,1


### songs.csv

The songs. Note that data is in unicode.

* song_id
* song_length: in ms
* genre_ids: genre category. Some songs have multiple genres and they are separated by |
* artist_name
* composer
* lyricist
* language

In [5]:
songs.head(6)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0
5,kKJ2JNU5h8rphyW21ovC+RZU+yEHPM+3w85J37p7vEQ=,235520,864|857|850|843,貴族精選,Joe Hisaishi,Hayao Miyazaki,17.0


### members.csv

user information.

* msno
* city
* bd: age. Note: this column has outlier values, please use your judgement.
* gender
* registered_via: registration method
* registration_init_time: format %Y%m%d
* expiration_date: format %Y%m%d

In [6]:
members.head(6)

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613
5,zgPOEyUn5a/Fvuzb3m69ajzxjkbblVtObglW89FzLdo=,13,43,female,9,20120703,20171006


## Data Processing

Now we have songs.csv, members.csv, train.csv and test1.csv, we want to reduce them into two tables: test and train. We could use left join operation to merge members and songs into train and test data, so that we have more feature columns to train. Besides, we need to remove the null data from the dataset.

To test the learned recommendation system in the end, we extract the true labels of test data into ytr

In [7]:
test = test1.drop(['target'],axis=1)
ytr = np.array(test1['target'])

rearrange columns of the test data so that it fits into remain codes.  

In [8]:
test_name = ['id','msno','song_id','source_system_tab',\
             'source_screen_name','source_type']
test['id']=np.arange(nts)
test = test[test_name]

extract song_cols feilds from songs.csv, then we do a left join with the training and test data, so that we could add more features to the evaluation.

In [9]:
print('Data preprocessing...')
song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

Data preprocessing...


Since the members registration time is not semantic, we could split it into three fields: year, month and date. 

In [10]:
members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['registration_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration_date'] = members['registration_init_time'].apply(lambda x: int(str(x)[6:8]))

same thing with the expiration date, we also need to drop the original registration time column.

In [11]:
members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['expiration_date'] = members['expiration_date'].apply(lambda x: int(str(x)[6:8]))
members = members.drop(['registration_init_time'], axis=1)

after we processed the members table, we do a left join again with the training and test data.

In [12]:
members_cols = members.columns
train = train.merge(members[members_cols], on='msno', how='left')
test = test.merge(members[members_cols], on='msno', how='left')

For null data in the train and test data, we fill them as -1.

In [13]:
train = train.fillna(-1)
test = test.fillna(-1)

We do some garbage collection work for members and songs

In [14]:
import gc
del members, songs; gc.collect();

In [15]:
cols = list(train.columns)
cols.remove('target')

Let's take a look at the current train data

In [16]:
train.head(6)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,artist_name,genre_ids,song_length,language,city,bd,gender,registered_via,expiration_date,registration_year,registration_month,registration_date,expiration_year,expiration_month
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,Bastille,359,206471,52.0,1,0,-1,7,5,2012,1,2,2017,10
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,Various Artists,1259,284584,52.0,13,24,female,9,11,2011,5,25,2017,9
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,Nas,1259,225396,52.0,13,24,female,9,11,2011,5,25,2017,9
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,Soundway,1019,255512,-1.0,13,24,female,9,11,2011,5,25,2017,9
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,Brett Young,1011,187802,52.0,1,0,-1,7,5,2012,1,2,2017,10
5,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3Hg5kugV1S0wzEVLAEfqjIV5UHzb7bCrdBRQlGygLvU=,explore,Explore,online-playlist,1,Desiigner,1259,247803,52.0,1,0,-1,7,5,2012,1,2,2017,10


then we will use LabelEncoder in sklearn to precess with the string data in train and test. We firstly get the total number of kinds in both train and test, then we use labelEncoder to put each kind a label.

In [17]:
for col in tqdm(cols):
    if train[col].dtype == 'object':
        train[col] = train[col].apply(str)
        test[col] = test[col].apply(str)

        le = LabelEncoder()
        train_vals = list(train[col].unique())
        test_vals = list(test[col].unique())
        le.fit(train_vals + test_vals)
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])

100%|██████████████████████████████████████████| 19/19 [00:00<00:00, 50.36it/s]


get the populartiy of the songs from the songs.csv and add it to the training data set.

In [18]:
# Song popularity
unique_songs = range(max(train['song_id'].max(), test['song_id'].max()))
song_popularity = pd.DataFrame({'song_id': unique_songs, 'popularity':0})

train_sorted = train.sort_values('song_id')
train_sorted.reset_index(drop=True, inplace=True)
test_sorted = test.sort_values('song_id')
test_sorted.reset_index(drop=True, inplace=True)

In [19]:
for unique_song in tqdm(unique_songs):
    if unique_song != (len(unique_songs)-1):
        train_pop = (train_sorted['song_id'].searchsorted(unique_song+1)[0] - 
                     train_sorted['song_id'].searchsorted(unique_song)[0])
        test_pop = (test_sorted['song_id'].searchsorted(unique_song+1)[0] - 
                     test_sorted['song_id'].searchsorted(unique_song)[0])
    else : 
        train_pop = (len(train_sorted) -
                     train_sorted['song_id'].searchsorted(unique_song)[0])
        test_pop = (len(test_sorted) -
                     test_sorted['song_id'].searchsorted(unique_song)[0])
    song_popularity[unique_song] = train_pop + test_pop

100%|█████████████████████████████████████| 5770/5770 [00:26<00:00, 218.65it/s]


## Model training and Prediction

Now we are going to train the model, note that we firstly split the training data into 90% training data and 10% validation data. 

In [20]:
# User library size
X = np.array(train.drop(['target'], axis=1))
y = train['target'].values

X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values

del train, test; gc.collect();

X_train, X_valid, y_train, y_valid = train_test_split(X, y, \
    test_size=0.1, random_state = 12)
    
del X, y; gc.collect();

d_train = lgb.Dataset(X_train, label=y_train)
d_valid = lgb.Dataset(X_valid, label=y_valid) 

watchlist = [d_train, d_valid]

Now we are going to train the model, we first create a parameter dict, set the learning rate to 0.4, and we make it print out the acurracy during the calculation. We also set the valid_sets to watchlist to validate our model.

In [21]:
print('Training LGBM model...')
params = {}
params['learning_rate'] = 0.4
params['application'] = 'binary'
params['max_depth'] = 15
params['num_leaves'] = 2**8
params['verbosity'] = 0
params['metric'] = 'auc'

model = lgb.train(params, train_set=d_train, num_boost_round=200, valid_sets=watchlist, \
early_stopping_rounds=10, verbose_eval=10)

Training LGBM model...
Training until validation scores don't improve for 10 rounds.
[10]	training's auc: 0.990018	valid_1's auc: 0.844558
[20]	training's auc: 0.999901	valid_1's auc: 0.839645
Early stopping, best iteration is:
[13]	training's auc: 0.997805	valid_1's auc: 0.849754


In [22]:
print('Making predictions and saving them...')
p_test = model.predict(X_test)

Making predictions and saving them...


In [23]:
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
subm.to_csv('submission.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')
print('Done!')

Done!


Now for each id in the test data, the model predicted the probablity that the user will listen to the corresponding song for the second time in the future. The result is stored in p_test. Now We use a hard thredhold rule to obtain yhat. That is, if $p_{test}[i]>0.5$, $yhat=1$, else $yhat=0$. Then we can calculate the accuracy acc of our lgbm model. 

In [24]:
yhat = (p_test>0.5).astype(int)
comp = (yhat==ytr).astype(int)
acc = comp.sum()/comp.size*100
print('The accuracy of lgbm model on test data is: {0:f}%'.format(acc))

The accuracy of lgbm model on test data is: 77.900000%


To make a comparison, we use a random guessing to predict on the same test data set. That is, to assign yhat_rand 1 or 2 for each id in test data according to $Bernoulli(1/2)$ distribution 

In [25]:
rd_seed = np.random.uniform(0,1,nts)
yhat_rand = (rd_seed>0.5).astype(int)
comp_rand = (yhat_rand==ytr).astype(int)
acc_rand = comp_rand.sum()/comp_rand.size*100
print('The accuracy of random model on test data is: {0:f}%'.format(acc_rand))

The accuracy of random model on test data is: 50.033333%


Obviously, lgbm model is better than random guessing. This means that the chosen lgbm model indeed improved the predicition accuracy on this problem. Of course, it is just a first trial, there are much work to do if one wants to further enhance the prediction accuracy.

# Reference

[1] Friedman J H. Greedy function approximation: a gradient boosting machine[J]. Annals of statistics, 2001: 1189-1232.

[2] Ke G, Meng Q, Wang T, et al. A Highly Efficient Gradient Boosting Decision Tree[C]//Advances in Neural Information Processing Systems. 2017: 3148-3156.

[3] https://www.kaggle.com/c/kkbox-music-recommendation-challenge

[4] https://github.com/Microsoft/LightGBM